<a href="https://colab.research.google.com/github/alengeorge-26/Plant-Disease-Classifier/blob/main/Plant_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [2]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt

In [21]:
IMAGE_SIZE =256
BATCH_SIZE=32
CHANNELS=3
EPOCHS=50
DIRECTORY="/content/drive/MyDrive/Plant_DIsease/PlantVillage"

In [ ]:
dataset=tf.keras.utils.image_dataset_from_directory(
  DIRECTORY,
   shuffle=True,
   image_size=(IMAGE_SIZE,IMAGE_SIZE),
   batch_size=BATCH_SIZE
)

In [ ]:
class_names=dataset.class_names

In [6]:
def get_dataset_partitions_tf(ds,train_split=.8,val_split=.1,test_split=.1,shuffle=True,shuffle_size=10000):
  
  ds_size=len(ds)

  if shuffle:
    ds=ds.shuffle(shuffle_size,seed=12)

  train_size=int(train_split*ds_size)
  val_size=int(val_split*ds_size)

  train_ds=ds.take(train_size)

  val_ds=ds.skip(train_size).take(val_size)
 
  test_ds=ds.skip(train_size).skip(val_size)
  
  return train_ds,val_ds,test_ds

In [7]:
train_ds,val_ds,test_ds=get_dataset_partitions_tf(dataset)

In [8]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [9]:
resize_and_rescale=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [10]:
data_augmentation=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [30]:
input_shape=(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)
n_classes=3

model=models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D((2,2,)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2,)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2,)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2,)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2,)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2,)),

    layers.Flatten(),
    
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax')
])

model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [32]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history=model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)